In [0]:
from keras.datasets import cifar10
import numpy as np

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [0]:

x_train = x_train.astype('float32') / 255.
x_test  = x_test .astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 32, 32, 3))
x_test  = np.reshape(x_test,  (len(x_test),  32, 32, 3))

In [0]:
from keras.layers import Input, Dense, Flatten, Reshape
from keras.models import Model

In [0]:
def create_dense_ae():
    # Dimension of coded image:
    encoding_dim = 150

    # encoder
    # Input placeholder
    input_img = Input(shape=(32, 32, 3))
    # Flatten
    flat_img = Flatten()(input_img)
    # encoded version of the image
    encoded = Dense(encoding_dim, activation='relu')(flat_img)
    
    # Decoder
    # Flattened layer af
    input_encoded = Input(shape=(encoding_dim,))
    flat_decoded = Dense(32*32*3, activation='sigmoid')(input_encoded)
    decoded = Reshape((32, 32, 3))(flat_decoded)

    # Model: 1st - Input layer, 2nd - Output layer
    encoder = Model(input_img, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_img, decoder(encoder(input_img)), name="autoencoder")
    return encoder, decoder, autoencoder

In [0]:
encoder, decoder, autoencoder = create_dense_ae()
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [0]:
autoencoder.summary()

In [0]:
autoencoder.fit(x_train, x_train,
                epochs=5,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

In [0]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

def plot_images(*args):
    args = [x.squeeze() for x in args]
    n = min([x.shape[0] for x in args])
    
    plt.figure(figsize=(2*n, 2*len(args)))
    for j in range(n):
        for i in range(len(args)):
            ax = plt.subplot(len(args), n, i*n + j + 1)
            plt.imshow(args[i][j])
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

    plt.show()

In [0]:
n = 10

imgs = x_test[:n]
encoded_imgs = encoder.predict(imgs, batch_size=n)
encoded_imgs[0]

In [0]:
decoded_imgs = decoder.predict(encoded_imgs, batch_size=n)

plot_images(imgs, decoded_imgs)

In [0]:
def noised_image(image,power_of_noise):
    noised_image = image + 3 * image.std() * np.random.random(image.shape)*power_of_noise
    return noised_image


In [0]:
plt.imshow(noised_image(x_test[29],0.1))

## Lab 8 task: 
### "Denoising Autoencoder"


1. You should create convolutional denoising autoencoder, which will remove noise from the images from CIFAR10 dataset.
2. Develop a function that takes as input noised image from CIFAR10 dataset and returns reconstructed one.
